In [1]:
# Preparations
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import seaborn as sns
from IPython.display import Latex
import warnings
from PrettyTable import PrettyTable
from functools import partial
from PrettyFigure import PrettyFigure
warnings.filterwarnings("ignore")
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['savefig.dpi'] = 75

# plt.rcParams['figure.autolayout'] = False
# plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.unicode'] = True
plt.rcParams['font.family'] = "STIX"
plt.rcParams['text.latex.preamble'] = "\\usepackage{subdepth}, \\usepackage{type1cm}"

results = {}

sns.set(color_codes=True)

def average(data):
    return 1 / len(data) * sum(data)

def error(data, average_of_data):
    s = sum([(x - average_of_data)**2 for x in data])
    return math.sqrt(s / (len(data) * (len(data) - 1)))

def std_deviation(error_of_average, length_of_dataset):
    return error_of_average * math.sqrt(length_of_dataset)

def average_with_weights(data, weights):
    d = data
    w = weights
    return (d * w**-2).sum() / (w**-2).sum()

def error_with_weights(weights):
    w = weights
    return 1 / math.sqrt((w**-2).sum())

def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    return (d * w**-2).sum() / (w**-2).sum()

def werr(group, weight_name):
    return 1 / math.sqrt((group[weight_name]**-2).sum())

In [2]:
# Constants
x0 = 1.0219 # [m]
e0 = 6e-2 # [m]

# Theoretische Grundlagen

In diesm Versuch wird ein horizontal gelagertes Federpendel unter harmonischer Anregung untersucht.
Durch den Wirbelstrombremseffekt wirkt eine geschwindigkeitsproportionale Reibkraft

\begin{equation}
F_{reib} = -\beta\cdot v
\label{eq:Freib}
\end{equation}

bremsend auf das Pendel. Wobei $\beta$ durch die Anzahl Magneten auf dem Pendel konstant gegeben ist.

Die Anregung kann mit

\begin{equation}
F_{anr}(t) = \hat{F}_e\cdot cos(\Omega t)
\label{eq:Fanr}
\end{equation}

dargestellt werden, wobei

\begin{equation}
\hat{F}_e = \hat{y}_e \cdot k_1
\label{eq:Fe}
\end{equation}

gilt.

## Gedämpfte Schwingung

Für ein freies geschwindigkeitsproportional gedämpftes Pendel gilt

\begin{equation}
y(t) = \hat{y}\cdot e^{-\Gamma t}cos(\omega t - \delta)
\label{eq:pendel}
\end{equation}

mit

<center>
$\Gamma=\frac{\beta}{2m}$  
$\omega=\sqrt{\omega^2_0-\Gamma^2}$  
$\omega^2_0 = \frac{K_{tot}}{m}$
$\Gamma$: Abklingkonstante  
$\omega$: Kreisfrequenz des gedaempften Pendels  
$\omega_0$: Kreisfrequenz des ungedaempften Pendels
</center>

## Erzwungene Schwingung

Die erzwungene Schwingung hat die Form

\begin{equation}
\hat{y}(\gamma) = \frac{k_1\cdot\hat{y}_e}{m\cdot\omega^2_0}\cdot\frac{1}{\sqrt{(1-\gamma^2)^2+4\gamma^2\alpha^2}}
\label{eq:pendelerzwungen}
\end{equation}

mit

<center>
$\gamma=\frac{\Omega}{\omega_0}=\frac{f_e}{f_0}$  
$\alpha=\frac{\Gamma}{\omega_0}=\frac{\Gamma}{2\pi f_0}$  
$\hat{y}_N = \hat{y}_0\cdot e^{-2\pi\alpha N}$  
$tan\delta=\frac{2\gamma\alpha}{1-\gamma^2}$
</center>

# Versuchsaufbau

Im folgenden sieht man das Fahrbahnpendel welches ausgemessen wird.
Es ist auf einer Fahrbahn gelagert, welche durch Druckluft den Gleiter anhebt. Oben und unten ist das Pendel mit zwei identischen Federn befestigt. Am oberen Ende ist der Gleiter durch ein Überesetzungsgestänge an einem Schrittmotor befestigt, welcher das Pendel anregt.
Zwei Lichtschranken registrieren die Nulldurchläufe des Pendels und des Schrittmotors.

![Versuchsaufbau](images/versuchsaufbau.png)

# Versuchsdurchführung

Zuerst wurde die Schwingungsdauer des Pendels bestimmt.
Dazu wurde das Pendel um verschiedene Distanzen $x_0$ ausgelenkt und dann losgelassen. Eine Lichtschranke registriert dabei jeden zweiten Nulldurchgang und stoppt dabei die Zeit.
Für jede Auslenkung $x_0$ wurde zweimal gemessen.
Die Triggerschwelle wurde dabei nicht zwischen 1-4 Volt gewählt wie im Versuchsbeschrieb empfohlen sondern nach der Empfehlung von Prof. Minamisawa auf 0.6 V eingestellt.

In [3]:
# Read Data
df = pd.read_csv('data/period.csv')
dfp = df

v1T = df['T'].mean()
v1sT = error(df['T'], v1T)

ax = df.plot(kind='scatter', x='x', y='T', label='gemessene Periode')
plt.axhline(v1T, label='mittlere Periodendauer', axes=ax, color='red')
plt.xlabel('x [m]')
plt.ylabel('T [s]')
# plt.ylim([0.005, 0.007])
plt.legend(bbox_to_anchor=(0.02, 0.98), loc=2, borderaxespad=0.2)

figure = PrettyFigure(
        plt.gcf(),
        label='fig:periodendauer',
        caption='Gemessene Periodendauer und die mittlere Periodendauer.'
)
plt.close()
figure.show()

Damit ergibt sich ein $T$ von {{'{:.2f}'.format(v1T)}} ± {{'{:.2f}'.format(v1sT)}}$s^{-1}$.

Um $\omega$ und $\Gamma$ zu berechnen wird das Pendel ausgelenkt und dann schwingen gelassen. Mit einem Fit an den Amplitudenverlauf können die gewünschten Werte durch den LSQ-Algotithmus ermittelt werden.

In [4]:
# Amplitudenfit

def pendulum(t, y, gamma, w, d, y0):
    return y * np.exp(-gamma * t) * np.cos(w * t - d) + y0

df = pd.DataFrame()
dft = None

for run in range(1, 6):
    dft = pd.read_csv('data/W6/3.2.{:}.txt'.format(run), sep='\t', skiprows=3, header=None, names=('t', 'y'))
    v, covar = curve_fit(pendulum, dft['t'], dft['y'])
    df = df.append(pd.DataFrame([np.concatenate((v, np.sqrt(np.diag(covar))[1:3]))], columns=('y', 'gamma', 'w', 'd', 'y0', 'sgamma', 'sw')))
    y_fit = [pendulum(t, v[0], v[1], v[2], v[3], v[4]) for t in dft['t']]

ax = dft.plot(kind='scatter', x='t', y='y', label='gemessene Auslenkung')
plt.plot(dft['t'], y_fit, axes=ax, label='Auslenkung mit Value Fitting', color='red')
plt.xlabel('Zeit [s]')
plt.ylabel('Auslenkung [m]')
plt.legend(bbox_to_anchor=(0.02, 0.98), loc=2, borderaxespad=0.2)
figure = PrettyFigure(
        plt.gcf(),
        label='fig:auslenkung',
        caption='Auslenkung des Pendels über Zeit nach initialer Auslenkung und Loslassen mit LSQ-Fit.'
)
plt.close()
figure.show()

df['w'] = np.abs(df['w'])
v2w0 = df['w0'] = np.sqrt(df['w']**2 + df['gamma'])
v2sw0 = np.sqrt(
               ((df['w']**2 + df['gamma'])**-1/2 + 2*df['w']) * df['sw']
              +((df['gamma']**2 + df['w'])**-1/2 + 2*df['gamma']) * df['sgamma']
             )

for index, row in df.iterrows():
    plt.plot(dft['t'], [pendulum(t, row['y'], row['gamma'], row['w'], row['d'], row['y0']) for t in dft['t']], label='Auslenkung mit Value Fitting')
plt.xlabel('Zeit [s]')
plt.ylabel('Auslenkung [m]')
plt.legend(bbox_to_anchor=(0.02, 0.98), loc=2, borderaxespad=0.2)
figure = PrettyFigure(
        plt.gcf(),
        label='fig:auslenkungen_fit',
        caption='Auslenkung des Pendels über Zeit bei verschiedenen initialen Auslenkungen und Loslassen gefittet.'
)
plt.close()
figure.show()
    
plt.figure()
plt.errorbar(np.arange(1,6), np.array(v2w0), yerr=np.array(v2sw0))
plt.xlim(0, 6)
plt.xlabel('Run \#')
plt.ylabel('$\omega_0 [s^{-1}]')
plt.legend(bbox_to_anchor=(0.02, 0.98), loc=2, borderaxespad=0.2)
figure = PrettyFigure(
        plt.gcf(),
        label='fig:w0_auslenkungen_fit',
        caption='Eigenfrequenz des Pendels errechnet aus den verschiedenen Runs.'
)
plt.close()
figure.show()
v2df = df

Wenn wir die Formel

<center>
$\omega=\sqrt{\omega^2_0-\Gamma^2}$
</center>

benutzen um $w_0$ zu ermitteln erhalten wir einen Mittelwert von {{'{:.2f}'.format(v2w0.mean())}} ± {{'{:.2f}'.format(v2sw0.mean())}} $s^{-1}$.

Im weiteren Verlauf des Versuches soll mit Hilfe von Amplituden und Phasenresonanz erneut $\omega_0$ bestimmt und mit dem aus der gedämpften Schwingung verglichen werden.

Da das Pendel zweiseitig mit identischen Federn eingespannt ist, ergibt sich $\omega^2_0 = \frac{2k}{m}$ mit $k = k_1$.

Damit kann \ref{eq:pendelerzwungen} auf

\begin{equation}
\hat{y}(\Omega) = \frac{\hat{y}_e}{2}\cdot\frac{1}{\sqrt{ (1-(\frac{\Omega}{\omega_0})^2)^2  + 4 (\frac{\Omega}{\omega_0})^2 (\frac{\Gamma}{\omega_0})^2}}
\label{eq:ydach}
\end{equation}

reduziert werden. Ein Fit wird nun im Folgenden gemacht.

In [5]:
def pendulum(W, y, gamma, w0):
    return y / 2 * 1 / ((1 - (W / w0)**2)**2 + 4 * (W/w0)**2 * (gamma/w0)**2)

dft = pd.read_csv('data/resonance.csv')
dft['W'] = 2 * math.pi / dft['T']
v3v, covar = curve_fit(pendulum, dft['W'], dft['A'], maxfev=10000)
v3err = np.sqrt(np.diag(covar))
y_fit = pendulum(np.linspace(0, 6, 3000), v3v[0], v3v[1], v3v[2])

ax = dft.plot(kind='scatter', x='W', y='A', label='gemessene Auslenkung unter Resonanz')
plt.plot(np.linspace(0, 6, 3000), y_fit, axes=ax, label='Auslenkung mit Value Fitting', color='red')
plt.xlabel('Erregerfrequenz [$s^{-1}$]')
plt.ylabel('Amplitude nach Einschwingvorgang [m]')
plt.legend(bbox_to_anchor=(0.02, 0.98), loc=2, borderaxespad=0.2)
figure = PrettyFigure(
        ax.figure,
        label='fig:auslenkung_erreg',
        caption='Amplituden nach Enschwingen bei verschiedenen Erregerfrequenzen.'
)
plt.close()
figure.show()

Aus dem Fit kann nun $\omega_0$ als {{'{:.2f}'.format(v3v[2])}} ± {{'{:.2f}'.format(v3err[2])}} $s^{-1}$ bestimmt werden.
Nun soll $\omega_0$ auch noch aus der Phasenresonanz bestimmt werden.

In [6]:
# Phase fit

def pendulum(W, gamma, w0, y0):
    x = (w0**2 - W**2) / np.sqrt((w0**2 - W**2)**2 + 4 * gamma**2 * W**2) + y0
    return np.arccos(x)

dft = pd.read_csv('data/resonance.csv')
dft['W'] = 2 * math.pi / dft['T']
v4v, covar = curve_fit(pendulum, dft['W'], dft['phi'], maxfev=10000)
v4err = np.sqrt(np.diag(covar))
y_fit = pendulum(np.linspace(0, 6, 3000), v4v[0], v4v[1], v4v[2])

ax = dft.plot(kind='scatter', x='W', y='phi', label='gemessene Auslenkung unter Resonanz')
plt.plot(np.linspace(0, 6, 3000), y_fit, axes=ax, label='Auslenkung mit Value Fitting', color='red')
plt.xlabel('Erregerfrequenz [$s^{-1}$]')
plt.ylabel('$\delta$ [rad]')
plt.legend(bbox_to_anchor=(0.02, 0.98), loc=2, borderaxespad=0.2)
figure = PrettyFigure(
        ax.figure,
        label='fig:auslenkung_phase',
        caption='Phase bei verschiedenen Erregerfrequenzen.'
)
plt.close()
figure.show()

Aus dem Fit kann nun $\omega_0$ als {{'{:.2f}'.format(v4v[1])}} ± {{'{:.2f}'.format(v4err[1])}} $s^{-1}$ bestimmt werden.

# Fehlerrechnung

Um erkennen zu können wie gut der Fit war wird mit jedem Fit eine Fehlertoleranz errechnet. Da be der Methode mit der gedämpften Schwingung jedoch nicht $\omega_0$ errechet wurde, wurde das Fehlerfortpflanzungsgesetz

<center>
    $s_{\omega_0} = \sqrt{ \frac{\partial\omega_0}{\partial\omega}\Big|_{\omega_0} \cdot s_\omega + \frac{\partial\omega_0}{\partial\Gamma}\Big|_{\omega_0} \cdot s_\Gamma }$
</center>

mit

<center>
    $\frac{\partial\omega_0}{\partial\omega} = (\omega^2 + \Gamma^2)^{-\frac{1}{2}} + 2\omega$  
    $\frac{\partial\omega_0}{\partial\Gamma} = (\omega^2 + \Gamma^2)^{-\frac{1}{2}} + 2\Gamma$
</center>

angewandt um den Fehler für $\omega_0$ zu berechnen.
Alle anderen Fehler wurden durch den Fit automatisch mitberechnet.
Da der systematische Fehler durch die hohe Messgenauigkeit des Distanzmessers und des Zeitmessers viel kleiner ist als der statistische, haben wir nur den statistischen Fehler zu beachten.
Die errechneten statistischen Fehler sind in den Resultaten tabellarisch dargestellt.

# Resultate

Im Folgenden sind die Resultate zur besseren Uebersicht noch einmal tabellarisch sowie grafisch gegenübergestellt.

In [7]:
labels = ['Gedämpfte Schwingung',
          'Amplitudenresonanz',
          'Phasenresonanz',
]
x = [
    v2w0.mean(),
    v3v[2],
    v4v[1],
]

xerr = [
    v2sw0.mean(),
    v3err[2],
    v4err[1],
]
y = [
    v2df['w'].mean(),
    0,
    0
]
yerr = [
    v2df['sw'].mean(),
    0,
    0
]
xstr = list(map(('{0:.2E}').format, x))
xerrstr = list(map(('{0:.2E}').format, xerr))
xerrrelstr = list(map(('{0:.2f}').format, np.array(xerr) / np.array(x) * 100))
ystr = list(map(('{0:.2E}').format, y))
yerrstr = list(map(('{0:.2E}').format, yerr))
data = PrettyTable(
    list(zip(labels, xstr, xerrstr, xerrrelstr, ystr, yerrstr)),
    caption='Die verschieden errechneten Eigenfrequenzen mit ihren Fehlern.',
    entries_per_column=len(xstr),
    extra_header=['Messart', 'w\_0 [1/s]', 's\_w\_0 [1/s]', 's\_w\_0 relativ [percent]', 'w [1/s]', 's\_w [1/s]'],
    significant_digits=2
)
data.show()

Messart,w\_0 [1/s],s\_w\_0 [1/s],s\_w\_0 relativ [percent],w [1/s],s\_w [1/s]
Gedämpfte Schwingung,4.45E+00,3.96E-02,0.89,4.44E+00,2.01E-04
Amplitudenresonanz,4.46E+00,2.37E-02,0.53,0.00E+00,0.00E+00
Phasenresonanz,4.74E+00,8.27E-02,1.74,0.00E+00,0.00E+00


In [8]:
y = np.linspace(1, 6, 3)
plt.errorbar(x, y, xerr=xerr, fmt='o')
plt.yticks(y, labels, rotation='horizontal')
plt.ylim((0, 7))
figure = PrettyFigure(
    plt.gcf(),
    label='fig:w0',
    caption='Die errechneten Eigenfrequenzen im Vergleich.'
)
plt.close()
figure.show()

In [9]:
labels = ['Gedämpfte Schwingung',
          'Amplitudenresonanz',
          'Phasenresonanz',
]
x = [
    v2df['gamma'].mean(),
    v3v[1],
    v4v[0],
]

xerr = [
    v2df['sgamma'].mean(),
    v3err[1],
    v4err[0],
]
xstr = list(map(('{0:.2E}').format, x))
xerrstr = list(map(('{0:.2E}').format, xerr))
xerrrelstr = list(map(('{0:.2f}').format, np.array(xerr) / np.array(x) * 100))
data = PrettyTable(
    
    list(zip(labels, xstr, xerrstr, xerrrelstr)),
    caption='Die verschieden errechneten Abklingkonstanten mit ihren Fehlern.',
    entries_per_column=len(xstr),
    extra_header=['Messart', 'Abklingkonstante [1/s]', 'Fehler [1/s]', 'relativer Fehler [percent]'],
    significant_digits=2
)
data.show()

Messart,Abklingkonstante [1/s],Fehler [1/s],relativer Fehler [percent]
Gedämpfte Schwingung,8.96E-02,1.98E-04,0.22
Amplitudenresonanz,2.14E-01,3.78E-02,17.67
Phasenresonanz,7.06E-01,1.42E-01,20.10


In [10]:
y = np.linspace(1, 6, 3)
plt.errorbar(x, y, xerr=xerr, fmt='o')
plt.yticks(y, labels, rotation='horizontal')
plt.ylim((0, 7))
figure = PrettyFigure(
    plt.gcf(),
    label='fig:gamma',
    caption='Die errechneten Abklingkonstanten im Vergleich.'
)
plt.close()
figure.show()

Alles in allem sehen die Ergebnisse sehr zufriedenstellend aus, wenn man die Zahlen zu $\omega_0$ betrachtet.
Bei der gedämpften Schwingung und der Amplitudenresonanz haben wir einen relatvien Fehler von 0.5% was extrem gut ist. Einzig bei der Phasenresonanz ist der Fehler 1.5% und der Wert ist auch etwas neben den anderen beiden Werten.
Wenn man jedoch die Abklingkonstante betrachtet, so ist der Fehler nur bei der Methode mit der gedämpften Schwingung im akzeptablen Bereich.
Man könnte hier einen gewichteten Mittelwert errechnen. Jedoch scheinen die Resultate für die verschidenen Methoden so daneben zu liegen, dass wohl die Messungen Fehlerbehaftet sind und noch einmal gemacht werden müssten.
Zuerst dachte ich dass ich einen Fehler mit der Zeitumrechnung beim Amplitudenmessgerät gemacht habe, jedoch weichen die Zeitunabhängigen Resultate ebenfalls voneinander ab, wodurch ich auf einen Messfehler tippe.

Wie man anhand der Resultate für $\omega$ und $\omega_0$ unleicht erkennen kann liegt der Unterschied innerhalb der Fehlertoleranzen, weswegen es mit diesem Gerät gar kein Unterschied gemessen werden kann.

# Messwerte

In [11]:
# Messwerte

data = PrettyTable(
    list(zip(dfp['x'], dfp['T'])),
    caption='Messwerte der Periode bei verschiedenen Anfangsauslenkungen.',
    entries_per_column=len(dfp['x'] / 2),
    extra_header=['x [m]', 'T [s]']
)
data.show()

data = PrettyTable(
    list(zip(dft['T'], dft['phi'], dft['A'])),
    caption='Messwerte der Phase bei verschiedenen Erregerfrequenzen.',
    entries_per_column=len(dft['T'] / 2),
    extra_header=['T [s]', 'phi [rad]','A [m]']
)
data.show()

x [m],T [s]
0.14,1.3417700000000001
0.14,1.34107
0.16,1.34218
0.16,1.34246
0.18,1.34276
0.18,1.34325
0.2,1.3427200000000001
0.2,1.34315
0.22,1.3423
0.22,1.34273


T [s],phi [rad],A [m]
1.1764700000000001,1.8847,0.007
1.2121,1.8758,0.01
1.24999,1.8746,0.013
1.29033,1.8744,0.019
1.33331,1.8326,0.031
1.3792799999999998,1.1137,0.044
1.42859,0.7202,0.071
1.48149,0.661,0.024
1.53915,0.588,0.016
1.7316200000000002,0.4438,0.011
